## 1. Import Essential Library

In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\deeplearning\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
if not os.path.exists('images'):
    os.mkdir('images')

## 2. Define hyper-parameters 

In [3]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
epochs = 20000
batch_size=128
sample_interval=50
colab = False

## 3. Load MNIST Dataset and Prepare the target annotations for GAN

In [4]:
# Load the dataset
(X_train, _), (_, _) = mnist.load_data()
# Rescale -1 to 1
X_train = X_train / 127.5 - 1.
X_train = np.expand_dims(X_train, axis=3)
# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

## 4. Lets define a function called build_generator() which defines our generator structure. Look at my slide.
- Dense: units=256, input_dim=latent_dim
- LeakyReLU: alpha=0.2
- BatchNormalization: momentum=0.8
- Dense: units=512
- LeakyReLU: alpha=0.2
- BatchNormalization: momentum=0.8
- Dense: units=1024
- LeakyReLU: alpha=0.2
- BatchNormalization: momentum=0.8
- Dense: units=number of pixel in each image in MNIST, actiovation = 'tanh'
- Reshape: shape = img_shape

In [6]:
def build_generator():
    model = Sequential()
    model.add(Dense(256,input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(img_rows*img_cols,activation='tanh'))
    model.add(Reshape(img_shape))
    model.summary()
    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

## 5. Lets define a function called build_discriminator() which defines our  discriminator structure
- Flatten: input_shape=img_shape
- Dense: units=512
- LeakyReLU: alpha=0.2
- Dense: units=256
- LeakyReLU: alpha=0.2
- Dense: units=1, activation = 'sigmoid'

In [7]:
def build_discriminator():

    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=.2))
    model.add(Dense(1,activation='sigmoid'))
    
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(inputs=img, outputs=validity)

## 6. Which optimizer do we use

In [8]:
optimizer = Adam(0.0002, 0.5)

## 7. In this section, we are trying to define a model and create it using compile.

In [9]:
# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


## 8. Generator definition not creation

In [10]:
# Build the generator
generator = build_generator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
__________

## 9. Define the combined model. A combination of discriminator and generator. Fill where question marks are located.

In [11]:
# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
# img = ??????????????????
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
validity = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

## 10. This is a tool for visualizing the performance of generator

In [12]:
def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    if(colab == False):
        fig.savefig("images/%d.png" % epoch)
    else:
        plt.show()
    plt.close()

## 11. Training loop

In [13]:
for epoch in range(epochs):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random batch of images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    # Generate a batch of new images
    gen_imgs = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(imgs,valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs,fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Train the generator (to have the discriminator label samples as valid)
    g_loss = combined.train_on_batch(noise,valid)

    # Plot the progress
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    # If at save interval => save generated image samples
    if epoch % sample_interval == 0:
        sample_images(epoch)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\deeplearning\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.700835, acc.: 57.81%] [G loss: 0.861571]
1 [D loss: 0.347825, acc.: 86.33%] [G loss: 0.908639]
2 [D loss: 0.298064, acc.: 89.84%] [G loss: 1.025495]
3 [D loss: 0.258961, acc.: 93.36%] [G loss: 1.137778]
4 [D loss: 0.220484, acc.: 97.66%] [G loss: 1.290637]
5 [D loss: 0.188392, acc.: 99.22%] [G loss: 1.366101]
6 [D loss: 0.176802, acc.: 98.44%] [G loss: 1.489456]
7 [D loss: 0.158062, acc.: 100.00%] [G loss: 1.586384]
8 [D loss: 0.127815, acc.: 100.00%] [G loss: 1.710991]
9 [D loss: 0.123055, acc.: 99.61%] [G loss: 1.794266]
10 [D loss: 0.117070, acc.: 100.00%] [G loss: 1.871067]
11 [D loss: 0.110157, acc.: 100.00%] [G loss: 1.876612]
12 [D loss: 0.092332, acc.: 100.00%] [G loss: 2.058310]
13 [D loss: 0.090105, acc.: 100.00%] [G loss: 2.083440]
14 [D loss: 0.081748, acc.: 100.00%] [G loss: 2.197556]
15 [D loss: 0.078384, acc.: 100.00%] [G loss: 2.250275]
16 [D loss: 0.070577, acc.: 100.00%] [G loss: 2.334655]
17 [D loss: 0.068197, acc.: 100.00%] [G loss: 2.404423]
18 [D loss

151 [D loss: 0.444387, acc.: 74.61%] [G loss: 2.156631]
152 [D loss: 0.246069, acc.: 87.50%] [G loss: 2.893137]
153 [D loss: 0.236120, acc.: 90.23%] [G loss: 2.828713]
154 [D loss: 0.334123, acc.: 82.03%] [G loss: 2.660465]
155 [D loss: 0.310347, acc.: 81.64%] [G loss: 2.853135]
156 [D loss: 0.548543, acc.: 73.05%] [G loss: 1.873213]
157 [D loss: 0.273936, acc.: 83.20%] [G loss: 2.494578]
158 [D loss: 0.235949, acc.: 90.62%] [G loss: 3.248083]
159 [D loss: 0.488893, acc.: 74.61%] [G loss: 2.259314]
160 [D loss: 0.245790, acc.: 86.33%] [G loss: 2.832284]
161 [D loss: 0.508015, acc.: 74.22%] [G loss: 2.357627]
162 [D loss: 0.251479, acc.: 87.50%] [G loss: 3.021448]
163 [D loss: 0.569922, acc.: 71.09%] [G loss: 2.046805]
164 [D loss: 0.287202, acc.: 83.59%] [G loss: 3.205646]
165 [D loss: 0.349962, acc.: 83.59%] [G loss: 3.258330]
166 [D loss: 0.400605, acc.: 79.69%] [G loss: 2.790373]
167 [D loss: 0.277896, acc.: 85.16%] [G loss: 3.162330]
168 [D loss: 0.515148, acc.: 75.78%] [G loss: 1.

301 [D loss: 0.700194, acc.: 44.92%] [G loss: 0.700758]
302 [D loss: 0.715529, acc.: 43.75%] [G loss: 0.686651]
303 [D loss: 0.713189, acc.: 43.75%] [G loss: 0.666500]
304 [D loss: 0.696959, acc.: 46.88%] [G loss: 0.699147]
305 [D loss: 0.704470, acc.: 45.31%] [G loss: 0.684917]
306 [D loss: 0.709586, acc.: 43.75%] [G loss: 0.663074]
307 [D loss: 0.697734, acc.: 47.27%] [G loss: 0.680685]
308 [D loss: 0.691216, acc.: 46.48%] [G loss: 0.694975]
309 [D loss: 0.711139, acc.: 43.75%] [G loss: 0.684204]
310 [D loss: 0.694427, acc.: 46.09%] [G loss: 0.677763]
311 [D loss: 0.708611, acc.: 44.92%] [G loss: 0.660975]
312 [D loss: 0.691731, acc.: 47.66%] [G loss: 0.666840]
313 [D loss: 0.703098, acc.: 44.14%] [G loss: 0.671028]
314 [D loss: 0.708216, acc.: 44.53%] [G loss: 0.662449]
315 [D loss: 0.702940, acc.: 45.31%] [G loss: 0.662309]
316 [D loss: 0.697234, acc.: 44.53%] [G loss: 0.658778]
317 [D loss: 0.698248, acc.: 44.92%] [G loss: 0.672617]
318 [D loss: 0.688924, acc.: 46.09%] [G loss: 0.

451 [D loss: 0.653666, acc.: 48.83%] [G loss: 0.694439]
452 [D loss: 0.654614, acc.: 49.61%] [G loss: 0.687555]
453 [D loss: 0.658222, acc.: 49.61%] [G loss: 0.692281]
454 [D loss: 0.650912, acc.: 48.83%] [G loss: 0.689283]
455 [D loss: 0.651557, acc.: 49.61%] [G loss: 0.695265]
456 [D loss: 0.654170, acc.: 50.00%] [G loss: 0.691112]
457 [D loss: 0.660537, acc.: 49.22%] [G loss: 0.690374]
458 [D loss: 0.663052, acc.: 48.05%] [G loss: 0.692187]
459 [D loss: 0.659447, acc.: 49.22%] [G loss: 0.688990]
460 [D loss: 0.660281, acc.: 49.61%] [G loss: 0.688708]
461 [D loss: 0.663244, acc.: 49.22%] [G loss: 0.694362]
462 [D loss: 0.656481, acc.: 50.78%] [G loss: 0.701137]
463 [D loss: 0.663722, acc.: 50.78%] [G loss: 0.700391]
464 [D loss: 0.660311, acc.: 49.22%] [G loss: 0.700271]
465 [D loss: 0.656643, acc.: 50.39%] [G loss: 0.702429]
466 [D loss: 0.651174, acc.: 49.61%] [G loss: 0.701467]
467 [D loss: 0.649049, acc.: 50.00%] [G loss: 0.706199]
468 [D loss: 0.652131, acc.: 50.00%] [G loss: 0.

...

19360 [D loss: 0.676417, acc.: 58.59%] [G loss: 0.883168]
19361 [D loss: 0.660940, acc.: 60.55%] [G loss: 0.823038]
19362 [D loss: 0.657862, acc.: 61.33%] [G loss: 0.879494]
19363 [D loss: 0.670891, acc.: 58.20%] [G loss: 0.856691]
19364 [D loss: 0.645131, acc.: 62.50%] [G loss: 0.873024]
19365 [D loss: 0.681505, acc.: 57.03%] [G loss: 0.863945]
19366 [D loss: 0.657285, acc.: 60.16%] [G loss: 0.864923]
19367 [D loss: 0.667535, acc.: 58.59%] [G loss: 0.854730]
19368 [D loss: 0.676214, acc.: 57.42%] [G loss: 0.860237]
19369 [D loss: 0.653777, acc.: 61.72%] [G loss: 0.876418]
19370 [D loss: 0.657894, acc.: 61.33%] [G loss: 0.859143]
19371 [D loss: 0.684804, acc.: 56.64%] [G loss: 0.852282]
19372 [D loss: 0.645207, acc.: 59.38%] [G loss: 0.835794]
19373 [D loss: 0.655002, acc.: 61.72%] [G loss: 0.830230]
19374 [D loss: 0.686864, acc.: 55.47%] [G loss: 0.878181]
19375 [D loss: 0.662585, acc.: 60.55%] [G loss: 0.836240]
19376 [D loss: 0.667448, acc.: 57.42%] [G loss: 0.844379]
19377 [D loss:

19510 [D loss: 0.687220, acc.: 48.44%] [G loss: 0.840899]
19511 [D loss: 0.654266, acc.: 60.55%] [G loss: 0.845335]
19512 [D loss: 0.660500, acc.: 61.72%] [G loss: 0.839559]
19513 [D loss: 0.682040, acc.: 55.47%] [G loss: 0.874578]
19514 [D loss: 0.651477, acc.: 61.33%] [G loss: 0.864416]
19515 [D loss: 0.674997, acc.: 56.64%] [G loss: 0.885906]
19516 [D loss: 0.670196, acc.: 60.16%] [G loss: 0.838771]
19517 [D loss: 0.696493, acc.: 59.38%] [G loss: 0.867593]
19518 [D loss: 0.666251, acc.: 57.81%] [G loss: 0.854298]
19519 [D loss: 0.670714, acc.: 55.86%] [G loss: 0.860183]
19520 [D loss: 0.674319, acc.: 58.20%] [G loss: 0.865845]
19521 [D loss: 0.699207, acc.: 55.47%] [G loss: 0.848643]
19522 [D loss: 0.680993, acc.: 57.81%] [G loss: 0.862139]
19523 [D loss: 0.687075, acc.: 53.12%] [G loss: 0.879483]
19524 [D loss: 0.671499, acc.: 54.69%] [G loss: 0.917292]
19525 [D loss: 0.665777, acc.: 60.16%] [G loss: 0.834397]
19526 [D loss: 0.660383, acc.: 60.55%] [G loss: 0.821836]
19527 [D loss:

19660 [D loss: 0.655027, acc.: 60.16%] [G loss: 0.850663]
19661 [D loss: 0.660809, acc.: 57.81%] [G loss: 0.892878]
19662 [D loss: 0.679556, acc.: 54.69%] [G loss: 0.832566]
19663 [D loss: 0.664063, acc.: 60.55%] [G loss: 0.835891]
19664 [D loss: 0.668105, acc.: 57.03%] [G loss: 0.834678]
19665 [D loss: 0.662938, acc.: 58.59%] [G loss: 0.877093]
19666 [D loss: 0.677689, acc.: 60.55%] [G loss: 0.872036]
19667 [D loss: 0.658679, acc.: 55.86%] [G loss: 0.837430]
19668 [D loss: 0.675023, acc.: 57.03%] [G loss: 0.839989]
19669 [D loss: 0.655430, acc.: 60.55%] [G loss: 0.813842]
19670 [D loss: 0.672081, acc.: 53.52%] [G loss: 0.870836]
19671 [D loss: 0.664274, acc.: 60.16%] [G loss: 0.855937]
19672 [D loss: 0.642703, acc.: 63.67%] [G loss: 0.862998]
19673 [D loss: 0.649314, acc.: 62.89%] [G loss: 0.862653]
19674 [D loss: 0.662016, acc.: 61.72%] [G loss: 0.894958]
19675 [D loss: 0.673953, acc.: 57.81%] [G loss: 0.874926]
19676 [D loss: 0.687351, acc.: 55.47%] [G loss: 0.866493]
19677 [D loss:

19810 [D loss: 0.682276, acc.: 55.08%] [G loss: 0.840807]
19811 [D loss: 0.682999, acc.: 55.47%] [G loss: 0.826278]
19812 [D loss: 0.667540, acc.: 56.64%] [G loss: 0.821220]
19813 [D loss: 0.692387, acc.: 52.73%] [G loss: 0.839282]
19814 [D loss: 0.653579, acc.: 64.06%] [G loss: 0.815117]
19815 [D loss: 0.635663, acc.: 62.89%] [G loss: 0.835674]
19816 [D loss: 0.657539, acc.: 56.64%] [G loss: 0.844140]
19817 [D loss: 0.670077, acc.: 59.38%] [G loss: 0.860063]
19818 [D loss: 0.670500, acc.: 57.81%] [G loss: 0.827372]
19819 [D loss: 0.674599, acc.: 57.03%] [G loss: 0.829366]
19820 [D loss: 0.634851, acc.: 63.28%] [G loss: 0.814736]
19821 [D loss: 0.646464, acc.: 60.55%] [G loss: 0.873809]
19822 [D loss: 0.690513, acc.: 54.69%] [G loss: 0.854637]
19823 [D loss: 0.676434, acc.: 54.69%] [G loss: 0.868214]
19824 [D loss: 0.639202, acc.: 60.94%] [G loss: 0.865333]
19825 [D loss: 0.685627, acc.: 56.25%] [G loss: 0.808027]
19826 [D loss: 0.665395, acc.: 56.64%] [G loss: 0.868246]
19827 [D loss:

19960 [D loss: 0.695327, acc.: 57.81%] [G loss: 0.859582]
19961 [D loss: 0.650349, acc.: 59.38%] [G loss: 0.853239]
19962 [D loss: 0.693907, acc.: 54.30%] [G loss: 0.840020]
19963 [D loss: 0.668453, acc.: 58.20%] [G loss: 0.813909]
19964 [D loss: 0.652741, acc.: 60.94%] [G loss: 0.880387]
19965 [D loss: 0.677278, acc.: 59.77%] [G loss: 0.877238]
19966 [D loss: 0.660049, acc.: 59.38%] [G loss: 0.877115]
19967 [D loss: 0.646204, acc.: 62.11%] [G loss: 0.882784]
19968 [D loss: 0.671616, acc.: 55.08%] [G loss: 0.847233]
19969 [D loss: 0.689792, acc.: 54.69%] [G loss: 0.834642]
19970 [D loss: 0.656842, acc.: 60.94%] [G loss: 0.825706]
19971 [D loss: 0.676888, acc.: 57.81%] [G loss: 0.844655]
19972 [D loss: 0.676215, acc.: 55.08%] [G loss: 0.840754]
19973 [D loss: 0.674400, acc.: 58.20%] [G loss: 0.862049]
19974 [D loss: 0.672324, acc.: 55.47%] [G loss: 0.851939]
19975 [D loss: 0.656922, acc.: 62.89%] [G loss: 0.873136]
19976 [D loss: 0.679299, acc.: 57.03%] [G loss: 0.853909]
19977 [D loss: